In [84]:
import csv
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib
from matplotlib import pyplot as plt
from sklearn import ensemble, cross_validation, learning_curve, metrics , tree, naive_bayes
from sklearn.tree import DecisionTreeRegressor
from sklearn.datasets import load_digits, load_breast_cancer
from sklearn.cross_validation import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
matplotlib.style.use('ggplot')
%matplotlib inline

In [85]:
#load data
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [86]:
y_train = df['label'].values
X_train = df.drop(labels='label', axis=1).values
X_test = df_test.values

In [88]:
%%time
#попробуем random forest
estimator = RandomForestClassifier(max_depth=50, n_estimators=100)
estimator.fit(X_train, y_train)

CPU times: user 25.4 s, sys: 179 ms, total: 25.6 s
Wall time: 25.6 s


In [89]:
%%time
#качество на кросс-валидации
print(cross_val_score(estimator, X_train, y_train, cv=5).mean())

[ 0.96311719  0.96298941  0.96225741  0.96522568  0.96462601]


In [91]:
xgb_estimator = xgb.XGBClassifier(max_depth=5, n_estimators=1000)

In [ ]:
%%time
xgb_estimator.fit(X_train, y_train)

In [ ]:
%%time
print(cross_val_score(xgb_estimator, X_train, y_train, cv=5).mean())

In [71]:
y_pred = estimator.predict(X_test)

In [82]:
with open('my_submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['ImageId', 'Label'])
    for i in range(y_pred.shape[0]):
        writer.writerow([i+1, y_pred[i]])